# Web Services
## Data on the web
* With the HTTP Request/Response well undestood and well supported, there was a natural move toward exchanging data between programs using these protocols
* We needed to come up with an agreed way to represent data going between applications and across networks
* There are two commonly used formats: XML and JSON
 * JSON: Simpler and easier
 * XML: Precise and descriptive

## XML
eXtensible Markup Language

Is a markup language

### Terminology:
 * tags: indicate the beginning and ending od elements
 * Attributes: Keyword/value pairs on the opening tag of XML
 * Serialize / De-Serialize: Convert data in one program into a common format that can be stored and/or transmitted between systems in a programming language-independent manner

### XML Basics:
 * Start tag
 * End tag
 * Text Content
 * Attribute
 * Self Closing Tag

Example:
```
 <person>
 <name>Chuck</name>
 <phone type='intl'>
 +1 734 303 4456
 </phone>
 <email hide='yes'/>
 </person>
 ```

 The spaces are only to do it readable

![xml tree](images/xml_tree.png)



## XML Schema
* Description of the legal format of an XML document
* Expressed in terms od constraints on the strcture and the content of documents
* Often used to specify a "contract" between systems - "My system will only accept XML that conforms to this particular Schema."
* If a particular piece of XML meets the specification of the Schema - it is said to "validate"
### XSD XML Schema (W3C spec)
* We will focus on the Worl Wide Web Consortium (W3C) version
* It is often called "W3C Schema" because "Schema" is considered generic.
* More commonly it is called XSD beacuse the file names end in .xsd

XSD Structure
 * xs:element
 * xs:sequence
 * xs:complexType

 ![xsd](images/xsd.png)

 Also we can specify:
 * Constraints,
 * Data Types,
 * Date/Time Format

In [16]:
# Parsing XML

import xml.etree.ElementTree as ET
data = '''
<person>
  <name>Chuck</name>
  <phone type="intl">
    +1 7 34 303 4456
  </phone>
  <email hide="yes"/>
</person>'''
tree = ET.fromstring(data)
print('Name:', tree.find('name').text)
print('Atrr:', tree.find('email').get('hide'))


Name: Chuck
Atrr: yes


In [17]:
# Looping through nodes

input = '''
<stuff>
  <users>
    <user x="2">
      <id>001</id>
      <name>Chuck</name>
    </user>
    <user x="7">
      <id>009</id>
      <name>Brent</name>
    </user>
  </users>
</stuff>'''

stuff = ET.fromstring(input)
lst = stuff.findall('users/user')
print('User count:', len(lst))

for item in lst:
    print('Name', item.find('name').text)
    print('Id', item.find('id').text)
    print('Attribute', item.get('x'))



User count: 2
Name Chuck
Id 001
Attribute 2
Name Brent
Id 009
Attribute 7


In [18]:
input = '''
<stuff>
  <users>
    <user x="2">
      <id>001</id>
      <name>Chuck</name>
    </user>
    <user x="7">
      <id>009</id>
      <name>Brent</name>
    </user>
  </users>
</stuff>'''

stuff = ET.fromstring(input)

lst = stuff.findall('users/user')
print('User count:', len(lst))

lst2 = stuff.findall('user')
print('User count:', len(lst2))

User count: 2
User count: 0


## JavaScript Object Notation - JSON

Represents data as nested "lists" and "dictionaries"


In [19]:
# Parsing JSON

import json

data = '''
[
  { "id" : "001",
    "x" : "2",
    "name" : "Chuck"
  } ,
  { "id" : "009",
    "x" : "7",
    "name" : "Brent"
  }
]'''

info = json.loads(data)
print('User count:', len(info))

for item in info:
    print('Name', item['name'])
    print('Id', item['id'])
    print('Attribute', item['x'])

User count: 2
Name Chuck
Id 001
Attribute 2
Name Brent
Id 009
Attribute 7


## Service Oriented Approach
* Most non-trivial web applications use services
* They use services from other apllications
 * Credit Card Charge
 * Hotel Reservation Systems
* Services publish the "rules" applications must follow to make use of the service (API)

### Multiple systems
* Initially two systems cooperate ans plit the problem
* As the data/service becomens useful - multiple applications want to use the information/application

## APIs
**Application Program Interface** itself is largely abstract in that it specifies an interface and controls the behavior of the objects specified in that interface. The software that provides the funtionality described by an APU is said to be an "implementation" of the API. An API is typically defined interms of the programming language used to build an application

In [6]:


import urllib.request, urllib.parse, urllib.error
import json
import ssl

api_key = False
# If you have a Google Places API key, enter it here
# api_key = 'AIzaSy___IDByT70'
# https://developers.google.com/maps/documentation/geocoding/intro

if api_key is False:
    api_key = 42
    serviceurl = 'http://py4e-data.dr-chuck.net/json?'
else :
    serviceurl = 'https://maps.googleapis.com/maps/api/geocode/json?'

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

while True:
    address = input('Enter location: ')
    #address = 'Ann Arbor, MI'
    if len(address) < 1: break

    parms = dict()
    parms['address'] = address
    if api_key is not False: parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters')

    try:
        js = json.loads(data)
    except:
        js = None

    if not js or 'status' not in js or js['status'] != 'OK':
        print('==== Failure To Retrieve ====')
        print(data)
        continue

    print(json.dumps(js, indent=4))

    lat = js['results'][0]['geometry']['location']['lat']
    lng = js['results'][0]['geometry']['location']['lng']
    print('lat', lat, 'lng', lng)
    location = js['results'][0]['formatted_address']
    print(location)

Retrieving http://py4e-data.dr-chuck.net/json?address=Ann+Arbor%2C+MI&key=42
Retrieved 1736 characters
{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "Ann Arbor",
                    "short_name": "Ann Arbor",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
                {
                    "long_name": "Washtenaw County",
                    "short_name": "Washtenaw County",
                    "types": [
                        "administrative_area_level_2",
                        "political"
                    ]
                },
                {
                    "long_name": "Michigan",
                    "short_name": "MI",
                    "types": [
                        "administrative_area_level_1",
                        "political"
                    ]
                },
                {
     

## API Security and Rate Limiting
* The compute resources to run these APIs are not "free"
* The data provided by these APIs is usually valuable
* The data providers might limit the number of request per day, demant an API "key", or even charge for usage
* They might change the rules as things progress

In [5]:
# # import urllib.request, urllib.parse, urllib.error
# # import twurl
# # import ssl

# # # https://apps.twitter.com/
# # # Create App and get the four strings, put them in hidden.py

# # TWITTER_URL = 'https://api.twitter.com/1.1/statuses/user_timeline.json'

# # # Ignore SSL certificate errors
# # ctx = ssl.create_default_context()
# # ctx.check_hostname = False
# # ctx.verify_mode = ssl.CERT_NONE

# # while True:
# #     print('')
# #     acct = input('Enter Twitter Account:')
# #     if (len(acct) < 1): break
# #     url = twurl.augment(TWITTER_URL,
# #                         {'screen_name': acct, 'count': '2'})
# #     print('Retrieving', url)
# #     connection = urllib.request.urlopen(url, context=ctx)
# #     data = connection.read().decode()
# #     print(data[:250])
# #     headers = dict(connection.getheaders())
# #     # print headers
# #     print('Remaining', headers['x-rate-limit-remaining'])

## Summary
* Service Oriented Architecture - allows an application to be broken into parts and distributed across a network
* An API is a contract for interaction
* Web Services provie infraestructure for applications cooperating (an API) over a network - SOAP and REST are two styles of web services
* XML and JSON are serialization formats